In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# authenticate
credential = DefaultAzureCredential()
# # Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="subscriptionid",
    resource_group_name="resourcegroup",
    workspace_name="workspacename",
)

In [2]:

import mltable
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(credential=DefaultAzureCredential())
data_asset = ml_client.data.get("sqlpurchasehistory", version="1")

tbl = mltable.load(data_asset.path)

df = tbl.to_pandas_dataframe()
df

Found the config file in: /config.json


,id,recent_purchases,zip,med_age
0,1,"1 Gallon Milk, 24 oz Bread, Dozen Eggs",35404,29
1,2,"16 oz Toothpaste, 12 oz Shampoo, 8 oz Soap",33956,64
2,3,"5 lb Potatoes, 3 lb Onions, 1 lb Carrots",59703,59
3,4,"2 lb Chicken, 1 lb Beef, 0.75 lb Salmon",73043,58
4,5,"18 oz Cereal, 6 oz Yogurt, 1.5 oz Granola Bars",75412,55
5,6,"16 oz Pasta, 24 oz Tomato Sauce, 3 cloves Garlic",15467,53
6,7,"Bunch of Bananas, 1 lb Grapes, 16 oz Strawberries",75217,28
7,8,"8 oz Chips, 16 oz Salsa, 12 oz Guacamole, 10 c...",65622,46
8,9,"6 Rolls Paper Towels, 12 Rolls Toilet Paper, 1...",60546,41
9,10,"1.5 qt Ice Cream, 12 inch Frozen Pizza, 16 oz ...",01002,23


In [3]:
import pandas as pd
import requests
import json

# Azure OpenAI API endpoint URL
endpoint_url = "Promptflow endpoint"

# API key or authentication token for accessing the endpoint
api_key = "prompt flow api key"

# Headers for the HTTP request
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

response_data = []

# Iterate through rows and make requests
for index, row in df.iterrows():
    input_purchases = row["recent_purchases"]
    input_age = row["med_age"]

    # Construct payload using the input text
    payload = {
        "purchase_history": input_purchases,
        "med_age": input_age,
        "max_tokens": 200
    }

    # Convert payload to JSON and encode as bytes
    payload_bytes = json.dumps(payload).encode("utf-8")

    # Send the POST request
    response = requests.post(endpoint_url, data=payload_bytes, headers=headers)

    # Process the response
    if response.status_code == 200:
        result = response.json()
        generated_text = result.get("output", "No generated text found")
        response_data.append({"Input":input_purchases, "Generated": generated_text})
    else:
        print(f"Request for '{input_purchases}' failed with status code:", response.status_code)

# Create a DataFrame from the response data list
response_df = pd.DataFrame(response_data)
response_df['id'] = df['id']

# Print the resulting DataFrame
response_df

,Input,Generated,id
0,"1 Gallon Milk, 24 oz Bread, Dozen Eggs",1. Cereal\n2. Butter\n3. Orange Juice,1
1,"16 oz Toothpaste, 12 oz Shampoo, 8 oz Soap",1. 16 oz Mouthwash\n2. Hair conditioner for ma...,2
2,"5 lb Potatoes, 3 lb Onions, 1 lb Carrots",1. Onion Chopper\n2. Potato Peeler\n3. Carrot ...,3
3,"2 lb Chicken, 1 lb Beef, 0.75 lb Salmon",1. Organic vegetables\n2. Grill accessories\n3...,4
4,"18 oz Cereal, 6 oz Yogurt, 1.5 oz Granola Bars",1. Fresh fruit smoothie\n2. 8 oz milk\n3. Whol...,5
5,"16 oz Pasta, 24 oz Tomato Sauce, 3 cloves Garlic",1. 1 lb Ground Beef\n2. Parmesan Cheese\n3. Fr...,6
6,"Bunch of Bananas, 1 lb Grapes, 16 oz Strawberries",1. Pineapple\n2. Watermelon\n3. Blueberries,7
7,"8 oz Chips, 16 oz Salsa, 12 oz Guacamole, 10 c...",1. 6 pack of craft beer\n2. 2 lb ground beef\n...,8
8,"6 Rolls Paper Towels, 12 Rolls Toilet Paper, 1...",1. Dishwashing liquid\n2. Hand soap\n3. All-pu...,9
9,"1.5 qt Ice Cream, 12 inch Frozen Pizza, 16 oz ...",1. Ice cream scoop \n2. Frozen dessert topping...,10


In [9]:
!pip install pyodbc


In [13]:
import pyodbc

# Define connection parameters
connection_parameters = {
    'server': 'sqlserver',
    'database': 'databasename',
    'username': 'username',
    'password': 'password',
    'driver': '{ODBC Driver 17 for SQL Server}',
}

# DataFrame to SQL data type mapping
# Define the dtype mapping dictionary
dtype_mapping = {
    'object': 'TEXT',
    'int64': 'INTEGER',
    'int32': 'INTEGER',  # Add this line to handle int32 dtype
    'float64': 'FLOAT'
}


# Function to generate CREATE TABLE statement from DataFrame columns
def generate_create_table_statement(df, table_name):
    columns = ',\n'.join([f"{col} {dtype_mapping[df[col].dtype.name]}" for col in df.columns])
    return f"CREATE TABLE {table_name} (\n{columns}\n);"

# Establish a connection to the database
conn = pyodbc.connect(
    f"DRIVER={connection_parameters['driver']};"
    f"SERVER={connection_parameters['server']};"
    f"DATABASE={connection_parameters['database']};"
    f"UID={connection_parameters['username']};"
    f"PWD={connection_parameters['password']}"
)

# Create a cursor
cursor = conn.cursor()

# Generate CREATE TABLE statement
create_table_statement = generate_create_table_statement(response_df, 'newtable')

# Create the table if it does not exist
cursor.execute(create_table_statement)

# Iterate through rows of the DataFrame and insert data into the database
for index, row in response_df.iterrows():
    # Example: Insert data into the table 'newtable'
    placeholders = ', '.join(['?'] * len(row))
    insert_query = f"INSERT INTO newtable VALUES ({placeholders})"
    cursor.execute(insert_query, tuple(row))

# Commit the transaction
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()
